Note book to explore solutions.

In [ ]:
import pandas as pd

data = [100.4, 102, 114, 200, 202]

series = pd.Series(data, index=['a', 'b', 'c', 'd', 'e'])

series.loc['e'] = 200

#print(series.iloc[2]) #access elements by index

#print(series.loc['e'])

#print(series[series >= 200])  # sort series, series are one colum rows

calories  = {}

Clean and remove null cells

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/movies_data_1.csv")
#df.head(10)
print(df.head(5))
#check null values

print(df.isnull().sum())

#this fills columns
#df.fillna(value=0, inplace=True)






#print(df.isnull().sum())
#print(df)
#print(df.info())
#print(df.shape)
#print(df.columns)

In [ ]:
# DATA CLEANING

#select all columns
allColumns = ['MOVIES','YEAR','GENRE','RATING','ONE-LINE','STARS','VOTES','RunTime','Gross','Extract_date','owner_company','nr_of_episodes']

#drop null columns
df.dropna(subset=allColumns, inplace=True)

#select only some columns
nColumns = ['GENRE','ONE-LINE','STARS']
ownerColumn = ['owner_company']

#clean where you find \n and ''space
for col in nColumns:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace('\n', '', regex=False)
            .str.strip()
        )
# to clean only in owner column
for col in ownerColumn:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace('\t', '', regex=False)
            .str.strip()
        )


In [ ]:
#COLUMN SPLITING

print(df['STARS'].head(10))

#df[['DIRECTOR','STARS']] = df['STARS'].str.split('|', n=1, expand=True)

#clean whitespace with str.strip()
df['DIRECTOR'] = df['DIRECTOR'].str.strip()
df['STARS-LIST'] = df['STARS-LIST'].str.strip()

print(df['DIRECTOR'].head(10))
print(df['STARS-LIST'].head(10))

#df.drop(columns=['STARS'], inplace=True)


In [87]:
print(df.head(10))

                                                MOVIES        YEAR  \
123                             The Hitman's Bodyguard      (2017)   
131                                      Jurassic Park      (1993)   
141                                      Don't Breathe      (2016)   
157  The Lord of the Rings: The Fellowship of the Ring      (2001)   
171                                        Escape Room  (I) (2019)   
174                                               Jaws      (1975)   
178                                       The Departed      (2006)   
189                                          Gone Girl      (2014)   
190                     Jumanji: Welcome to the Jungle      (2017)   
191                                    American Psycho      (2000)   

                         GENRE  RATING  \
123      Action, Comedy, Crime     6.9   
131  Action, Adventure, Sci-Fi     8.1   
141    Crime, Horror, Thriller     7.1   
157   Action, Adventure, Drama     8.8   
171  Action, Advent

In [ ]:
df['Extract_date'] = pd.to_datetime(df['Extract_date'],errors='coerce') #to_datetime converts it to a object

#create column from extact date and time
df['extraction_date'] = df['Extract_date'].dt.date
df['extraction_time'] = df['Extract_date'].dt.time

print(df[['extraction_time','extraction_date']].head())

df.drop(columns=['Extract_date'], inplace=True)


In [ ]:
#YEAR CALCULATIONS

#convert to string
df['YEAR'] = df['YEAR'].astype(str)


#clean unwanted character
df['YEAR'] = df['YEAR'].str.strip()

df['start_year'] = s

df[[]]




In [ ]:
#to check if data is changed correctly

from datetime import datetime

timeStamp = datetime.now().strftime("%H%M%S")

df.to_csv(f"../output/outputAtTime{timeStamp}.csv",index=False)
#df.to_csv(f"../output/outputDataCleaned.csv",index=False)